#대회에서 제공하는 데이터는 게임 플레이어의 행동 정보를 담고 있습니다. 이 데이터를 사용하여 게임에서 승리하는 선수를 예측합니다. 데이터는 5만여 개의 경기 리플레이 데이터로 이루어져 있으며, 각 리플레이 데이터는 총 경기 시간의 일부에 대한 인게임 정보를 포함합니다.



train.csv / test.csv

game_id : 경기 구분 기호

winner : player 1의 승리 확률

time : 경기 시간, 마침표(.)로 분과 초가 구분됩니다. ex) 2.24 = 2분 24초  

player : 선수

##1) 0: player 0

##2) 1: player 1

species : 종족

##1) T: 테란

##2) P: 프로토스

##3) Z: 저그

event : 행동 종류

event_contents : 행동 상세

##1) Ability : 생산, 공격 등 선수의 주요 행동

##2) AddToControlGroup : 부대에 추가

##3) Camera : 시점 선택

##4) ControlGroup : 부대 행동

##5) GetControlGroup : 부대 불러오기

##6) Right Click : 마우스 우클릭

##7) Selection : 객체 선택

##8) SetControlGroup : 부대 지정

 

sample_submission.csv

game_id : 경기 구분 기호

winner : player 1의 승리 확률


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls '/content/drive/'


'My Drive'


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
train = pd.read_csv("/content/drive/My Drive/scft/train.csv", encoding = 'utf-8')
test = pd.read_csv("/content/drive/My Drive/scft/test.csv", encoding = 'utf-8')

In [ ]:
#@title
train_Ability = train[train['event'] == 'Ability']
#train_Ability_T = train_Ability[train_Ability['species'] == 'T']
#train_Ability_T.event_contents.value_counts()
#(1360) - TrainSCV                                                        
#(13A0) - LowerSupplyDepot  #보급고                                         
#MULE(지게로봇, Mobile Utility Lunar Excavator)
#a = train_Ability_T
#일꾼수만 일단 계산해보자
#game_id == 0
#b = a[['game_id', 'winner', 'player', 'species', 'event_contents']]
#c = a[['game_id', 'winner', 'player', 'species']]
#train_Ability_T.event_contents.value_counts()

In [ ]:
#테프저 모두 일꾼을 worker로 변환
t = train_Ability.replace({'event_contents': '(1360) - TrainSCV'}, {'event_contents': 'worker'})
tz = t.replace({'event_contents': '(1820) - MorphDrone'}, {'event_contents': 'worker'})
train_worker = tz.replace({'event_contents': '(15E0) - TrainProbe'}, {'event_contents': 'worker'})
worker = train_worker[train_worker['event_contents'] == 'worker']
worker

,game_id,winner,time,player,species,event,event_contents
3,0,1,0.02,0,T,Ability,worker
29,0,1,0.33,1,T,Ability,worker
37,0,1,0.35,0,T,Ability,worker
63,0,1,0.41,1,T,Ability,worker
101,0,1,1.03,0,T,Ability,worker
...,...,...,...,...,...,...,...
67091343,38871,0,7.34,0,Z,Ability,worker
67091434,38871,0,7.47,0,Z,Ability,worker
67091606,38871,0,8.19,0,Z,Ability,worker
67091682,38871,0,8.37,0,Z,Ability,worker


In [ ]:
player_0 = []
for i in range(0,38872):
  worker_i = worker[(worker['game_id']== i) & (worker['player'] == 0)]
  a = len(worker_i)
  player_0.append(a)

In [ ]:
#@title
player_1 = []
for i in range(0,38872):
  worker_i = worker[(worker['game_id']== i) & (worker['player'] == 1)]
  a = len(worker_i)
  player_1.append(a) 

In [ ]:
P0_worker = pd.DataFrame(player_0)         
P1_worker = pd.DataFrame(player_1)
worker.pd = pd.concat([P0_worker, P1_worker], axis=1)
worker.pd.columns = ['P0_worker','P1_worker']
worker.pd['delta_worker'] = worker.pd['P0_worker'] - worker.pd['P1_worker']
worker.pd

In [ ]:
train_yes = pd.read_pickle('/content/drive/My Drive/scft/train_yes.pkl')
train_yes

In [ ]:
train_yesman = pd.concat([train_yes,worker.pd ], axis=1)

In [ ]:
train_yesman

,P0_starting,P1_starting,Map,P0_species,P0_Ability,P0_AddToControlGroup,P0_Camera,P0_ControlGroup,P0_GetControlGroup,P0_Right Click,P0_Selection,P0_SetControlGroup,P1_species,P1_Ability,P1_AddToControlGroup,P1_Camera,P1_ControlGroup,P1_GetControlGroup,P1_Right Click,P1_Selection,P1_SetControlGroup,delta_Ability,delta_AddToControlGroup,delta_Camera,delta_ControlGroup,delta_GetControlGroup,delta_Right Click,delta_Selection,delta_SetControlGroup,P0_species_POW,P1_species_POW,delta_species_POW,P0_m_species_POW,P1_m_species_POW,delta_m_species_POW,P0_m_starting_species_POW,P1_m_starting_species_POW,delta_m_starting_species_POW,P0_worker,P1_worker,delta_worker
0,2.0,6.0,1.0,0,34.0,2.0,444.0,0.0,24.0,35.0,50.0,3.0,0,34.0,0.0,425.0,0.0,3.0,28.0,57.0,1.0,0.0,2.0,19.0,0.0,21.0,7.0,-7.0,2.0,50.00,50.00,0.00,50.00,50.00,0.00,51.74,48.26,3.48,9,6,3
1,3.0,4.0,2.0,1,77.0,1.0,627.0,0.0,162.0,160.0,186.0,10.0,0,67.0,0.0,858.0,0.0,31.0,131.0,116.0,8.0,10.0,1.0,-231.0,0.0,131.0,29.0,70.0,2.0,50.12,49.88,0.24,49.94,50.06,-0.12,46.99,53.01,-6.02,22,17,5
2,13.0,1.0,0.0,1,69.0,6.0,413.0,0.0,99.0,160.0,90.0,14.0,2,85.0,5.0,725.0,2.0,109.0,204.0,232.0,9.0,-16.0,1.0,-312.0,-2.0,-10.0,-44.0,-142.0,5.0,48.79,51.21,-2.42,47.13,52.87,-5.74,52.45,47.55,4.90,19,23,-4
3,8.0,12.0,5.0,0,82.0,0.0,713.0,0.0,132.0,276.0,180.0,6.0,1,89.0,0.0,388.0,0.0,710.0,268.0,148.0,19.0,-7.0,0.0,325.0,0.0,-578.0,8.0,32.0,-13.0,49.88,50.12,-0.24,51.58,48.42,3.16,51.75,48.25,3.50,7,24,-17
4,10.0,9.0,6.0,0,57.0,1.0,430.0,0.0,224.0,177.0,67.0,10.0,2,36.0,4.0,272.0,0.0,99.0,106.0,126.0,8.0,21.0,-3.0,158.0,0.0,125.0,71.0,-59.0,2.0,49.35,50.65,-1.30,47.63,52.37,-4.74,47.07,52.93,-5.86,14,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38867,6.0,2.0,1.0,0,43.0,0.0,639.0,0.0,0.0,95.0,53.0,1.0,1,32.0,0.0,324.0,0.0,0.0,43.0,58.0,0.0,11.0,0.0,315.0,0.0,0.0,52.0,-5.0,1.0,49.88,50.12,-0.24,52.78,47.22,5.56,53.29,46.71,6.58,11,6,5
38868,4.0,3.0,2.0,0,63.0,0.0,558.0,0.0,0.0,123.0,62.0,0.0,1,28.0,0.0,633.0,0.0,8.0,56.0,48.0,3.0,35.0,0.0,-75.0,0.0,-8.0,67.0,14.0,-3.0,49.88,50.12,-0.24,50.06,49.94,0.12,54.06,45.94,8.12,7,11,-4
38869,13.0,1.0,0.0,1,56.0,0.0,785.0,0.0,28.0,248.0,106.0,4.0,2,53.0,2.0,330.0,0.0,89.0,167.0,191.0,6.0,3.0,-2.0,455.0,0.0,-61.0,81.0,-85.0,-2.0,48.79,51.21,-2.42,47.13,52.87,-5.74,52.45,47.55,4.90,17,13,4
38870,7.0,15.0,4.0,1,20.0,1.0,321.0,0.0,11.0,28.0,24.0,1.0,1,36.0,0.0,385.0,0.0,43.0,60.0,64.0,5.0,-16.0,1.0,-64.0,0.0,-32.0,-32.0,-40.0,-4.0,50.00,50.00,0.00,50.00,50.00,0.00,50.60,49.40,1.20,9,12,-3


In [ ]:
train_Ability = train[train['event'] == 'Ability']
train_Ability_T = train_Ability[train_Ability['species'] == 'T']
train_Ability_T.event_contents.value_counts()

(1360) - TrainSCV                                                             350376
(13E0) - TrainMarine                                                          134801
(13A0) - LowerSupplyDepot                                                      42761
(1C60) - UpgradeToOrbitalCommand                                               34531
(1401) - BuildSiegeTank                                                        19363
                                                                               ...  
(1160) - ScannerSweep; Location: (33.54638671875, 114.038818359375, 40944)         1
(1020) - BuildCommandCenter; Location: (127.5, 22.5, 49119)                        1
(5A0) - Attack; Location: (32.918701171875, 115.7587890625, 40935)                 1
(5A0) - Attack; Location: (42.809814453125, 140.901123046875, 49127)               1
(1023) - BuildBarracks; Location: (77.5, 118.5, 40941)                             1
Name: event_contents, Length: 421013, dtype: int64

In [ ]:
train_Ability_Z = train_Ability[train_Ability['species'] == 'Z']
train_Ability_Z.event_contents.value_counts()
#(1820) - MorphDrone
#(1822) - MorphOverlord

(1820) - MorphDrone                                                     287254
(1822) - MorphOverlord                                                   90700
(1821) - MorphZergling                                                   83416
(1E60) - TrainQueen                                                      51130
(1829) - MorphRoach                                                      18200
                                                                         ...  
(2120) - BuildCreepTumor; Location: (145.5, 119.5, 40941)                    1
(5A0) - Attack; Location: (137.217041015625, 41.753173828125, 48145)         1
(5A0) - Attack; Location: (112.86328125, 67.26513671875, 32428)              1
(5A0) - Attack; Location: (149.7802734375, 41.21044921875, 40941)            1
(5A0) - Attack; Location: (157.142578125, 80.02197265625, 40941)             1
Name: event_contents, Length: 225505, dtype: int64

In [ ]:
train_Ability_P = train_Ability[train_Ability['species'] == 'P']
train_Ability_P.event_contents.value_counts()
#(15E0) - TrainProbe 

(15E0) - TrainProbe                                                                          258765
(1C80) - TransformToWarpGate                                                                  16109
(1586) - TrainAdept                                                                           15566
(1581) - TrainStalker                                                                         14134
(1D86) - ResearchWarpGate                                                                     12521
                                                                                              ...  
(5A0) - Attack; Location: (17.085693359375, 87.704345703125, 49122)                               1
(5A0) - Attack; Target: Probe [03240001]; Location: (24.2578125, 150.369873046875, 49120)         1
(3380) - Revelation; Location: (28.015625, 22.42529296875, 49105)                                 1
(1546) - BuildTwilightCouncil; Location: (57.5, 25.5, 49119)                                      1


In [ ]:
#인덱싱을 통해서 추출
train_RC = train[train['event'] == 'Right Click']
train_rc = train_RC.reset_index()
train_rc1 = train_rc['event_contents']
train_rc1 #순서는 같으니깐 나중에 게임아이디 붙여넣으면 될듯

0           Location: (136.3759765625, 41.11962890625, 49016)
1           Location: (136.033935546875, 41.735107421875, ...
2           Location: (32.222900390625, 153.720703125, 49120)
3           Location: (29.195068359375, 143.441650390625, ...
4           Location: (29.195068359375, 129.53466796875, 4...
                                  ...                        
11565872    Target: DestructibleSignsFunny [004C0001]; Loc...
11565873    Target: DestructibleSignsFunny [004C0001]; Loc...
11565874    Target: DestructibleSignsFunny [004C0001]; Loc...
11565875    Location: (136.720458984375, 69.50830078125, 4...
11565876    Location: (138.084228515625, 67.595703125, 40907)
Name: event_contents, Length: 11565877, dtype: object

In [ ]:
RCLIST = []
for i in range(0,11565877):
  rc_i = train_rc1[i]
  a = rc_i[0]
  RCLIST.append(a)

In [ ]:
train_rcc = train_rc[['game_id', 'event_contents']]
train_rc

,index,game_id,winner,time,player,species,event,event_contents
0,17,0,1,0.20,0,T,Right Click,"Location: (136.3759765625, 41.11962890625, 49016)"
1,18,0,1,0.23,0,T,Right Click,"Location: (136.033935546875, 41.735107421875, ..."
2,111,0,1,1.17,0,T,Right Click,"Location: (32.222900390625, 153.720703125, 49120)"
3,112,0,1,1.18,0,T,Right Click,"Location: (29.195068359375, 143.441650390625, ..."
4,113,0,1,1.18,0,T,Right Click,"Location: (29.195068359375, 129.53466796875, 4..."
...,...,...,...,...,...,...,...,...
11565872,67091737,38871,0,8.48,0,Z,Right Click,Target: DestructibleSignsFunny [004C0001]; Loc...
11565873,67091741,38871,0,8.48,0,Z,Right Click,Target: DestructibleSignsFunny [004C0001]; Loc...
11565874,67091744,38871,0,8.48,0,Z,Right Click,Target: DestructibleSignsFunny [004C0001]; Loc...
11565875,67091765,38871,0,8.50,0,Z,Right Click,"Location: (136.720458984375, 69.50830078125, 4..."


In [ ]:
event_contents_n = pd.DataFrame(RCLIST) 
#event_contents_n.columns('event_contents_n')
rightclick = pd.concat([train_rc,event_contents_n], axis=1)
rc = rightclick[['game_id', 'winner', 'player',0]]
rc.columns = ['game_id','winner','player','rightclick_contents']

,game_id,winner,player,rightclick_contents
0,0,1,0,L
1,0,1,0,L
2,0,1,0,L
3,0,1,0,L
4,0,1,0,L
...,...,...,...,...
11565872,38871,0,0,T
11565873,38871,0,0,T
11565874,38871,0,0,T
11565875,38871,0,0,L


In [ ]:
#스플릿함수로 뽑아냄


In [ ]:
p0L = []
p0T = []
p1L = []
p1T = []
for i in range(0, 38872):
  rc_i = rc[rc['game_id'] == i]
 
  a = rc_i[(rc_i['player'] == 0) & (rc_i['rightclick_contents'] == 'L')]
  b = rc_i[(rc_i['player'] == 0) & (rc_i['rightclick_contents'] == 'T')]
  p0L.append(len(a))
  p0T.append(len(b))

  a = rc_i[(rc_i['player'] == 1) & (rc_i['rightclick_contents'] == 'L')]
  b = rc_i[(rc_i['player'] == 1) & (rc_i['rightclick_contents'] == 'T')]
  p1L.append(len(a))
  p1T.append(len(b))

In [ ]:
p0_Location = pd.DataFrame(p0L)         
p0T_Target = pd.DataFrame(p0T)
p1L_Location = pd.DataFrame(p1L)         
p1T_Target = pd.DataFrame(p1T)

In [ ]:
Click_Location = pd.concat([p0_Location, p1L_Location], axis=1)
Click_Target = pd.concat([p0T_Target, p1T_Target], axis=1)

In [ ]:
Click_Location.columns = ['P0_rc_locaction','P1_rc_location']
Click_Location['delta_rc_location'] =Click_Location['P0_rc_locaction'] - Click_Location['P1_rc_location']
Click_Target.columns = ['P0_rc_target','P1_rc_target']
Click_Target['delta_rc_target'] = Click_Target['P0_rc_target'] - Click_Target['P1_rc_target']

In [ ]:
right_click_L_T = pd.concat([Click_Location, Click_Target], axis=1)
right_click_L_T

,P0_rc_locaction,P1_rc_location,delta_rc_location,P0_rc_target,P1_rc_target,delta_rc_target
0,24,11,13,11,17,-6
1,119,116,3,41,15,26
2,116,133,-17,44,71,-27
3,190,143,47,86,125,-39
4,132,97,35,45,9,36
...,...,...,...,...,...,...
38867,73,24,49,22,19,3
38868,113,39,74,10,17,-7
38869,180,133,47,68,34,34
38870,19,50,-31,9,10,-1


In [ ]:
x_n_train = pd.concat([train_yesman, right_click_L_T], axis=1)

In [ ]:
x_n_train.to_pickle('x_n_train.pkl')

In [ ]:
x_n_train.index.name = 'game_id'

In [ ]:
x_train = x_n_train

In [ ]:
#스타팅 포인트가 어디냐에 따른 승률
#특정 맵에서 스타팅 포인트가 어디냐에 따른 종족간 승률
#어빌리티 일꾼 생산 나머지 데이터 삭제 또는 인원생산 건물 건설
#밸류카운트


In [ ]:
#카메라 좌표 - 분포
#좌표 겹치는 부분으로 (카메라) - 활동 내역
#케글 스타크래프트
#파생변수 만들기
#스타팅 포인트
#승리하기 위해서는 용병을 많이 생산하고 싸워서 이겨서 건물을 부숴야한다.
#그렇다면 어빌리티 중 공격과 생산이 있는데 공격 한다면 무조건적으로 방어를 해야한다. 
#그러면 플레이어 1과 0이 공격을 한상황에서 발생한 어빌리티를 제외하고 나머지 생산을 잘할 수록 경기에 유리하기에 상대적 어빌리티 양으로 승리에 영향을 줄 수 있다.
#시점이 주로 상대방에 있을때 승리할 확률이 높다
#그 이유는 주로 공격을 하기 때문이다.
#경기 후반대에부대지정이 많을 수록 다양한 전술이 나올 수 있다.
#시야확보가 가장 중요 왜냐하면 상대방이 어떤 빌드 어떤 전략을 준비하는지 알고 대처할 수 있기때문.
#정찰필수
#클릭 로케이션 어택
#베이지안 옵티마이제이션
#일꾼생산수
#맵마다 스타팅포인트마다 종족간 승률
#우클릭 로케이션 타겟
#인구수 늘리는 건물생산수
#시야확보
#유닛업그레이드
#커맨드 센터 수
#trainscv 전체 게임에서 다 동일한 1360으로 표현
#threading 공부하기

In [ ]:
#내가 가지고 있는 정보
#종족전별 승률
#게임아이디 플레이어 승리자 종족 행동 스타팅포인트 맵
#종족, 행동, 스타팅포인트 맵 4가지를 이용해 파생변수 생성해야할듯
#event_content 컬럼은 결측치가 많고 전수조사가 아닌 만큼 빠지는 내용이 많다. 그리고 디테일하지 않다.
#데이콘 베이스라인 참조
#pip
#모델을 스코어로 변경
#프로그래머스
 

In [ ]:
xy_train = pd.read_pickle('/content/drive/My Drive/scft/xy_train.pkl')
xy_train

,P0_species,P0_Ability,P0_AddToControlGroup,P0_Camera,P0_ControlGroup,P0_GetControlGroup,P0_Right Click,P0_Selection,P0_SetControlGroup,P1_species,P1_Ability,P1_AddToControlGroup,P1_Camera,P1_ControlGroup,P1_GetControlGroup,P1_Right Click,P1_Selection,P1_SetControlGroup,delta_Ability,delta_AddToControlGroup,delta_Camera,delta_ControlGroup,delta_GetControlGroup,delta_Right Click,delta_Selection,delta_SetControlGroup
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,34.0,2.0,444.0,0.0,24.0,35.0,50.0,3.0,0,34.0,0.0,425.0,0.0,3.0,28.0,57.0,1.0,0.0,2.0,19.0,0.0,21.0,7.0,-7.0,2.0
1,1,77.0,1.0,627.0,0.0,162.0,160.0,186.0,10.0,0,67.0,0.0,858.0,0.0,31.0,131.0,116.0,8.0,10.0,1.0,-231.0,0.0,131.0,29.0,70.0,2.0
2,1,69.0,6.0,413.0,0.0,99.0,160.0,90.0,14.0,2,85.0,5.0,725.0,2.0,109.0,204.0,232.0,9.0,-16.0,1.0,-312.0,-2.0,-10.0,-44.0,-142.0,5.0
3,0,82.0,0.0,713.0,0.0,132.0,276.0,180.0,6.0,1,89.0,0.0,388.0,0.0,710.0,268.0,148.0,19.0,-7.0,0.0,325.0,0.0,-578.0,8.0,32.0,-13.0
4,0,57.0,1.0,430.0,0.0,224.0,177.0,67.0,10.0,2,36.0,4.0,272.0,0.0,99.0,106.0,126.0,8.0,21.0,-3.0,158.0,0.0,125.0,71.0,-59.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38867,0,43.0,0.0,639.0,0.0,0.0,95.0,53.0,1.0,1,32.0,0.0,324.0,0.0,0.0,43.0,58.0,0.0,11.0,0.0,315.0,0.0,0.0,52.0,-5.0,1.0
38868,0,63.0,0.0,558.0,0.0,0.0,123.0,62.0,0.0,1,28.0,0.0,633.0,0.0,8.0,56.0,48.0,3.0,35.0,0.0,-75.0,0.0,-8.0,67.0,14.0,-3.0
38869,1,56.0,0.0,785.0,0.0,28.0,248.0,106.0,4.0,2,53.0,2.0,330.0,0.0,89.0,167.0,191.0,6.0,3.0,-2.0,455.0,0.0,-61.0,81.0,-85.0,-2.0


In [ ]:
winners = train.groupby(['game_id']).winner.max()

In [ ]:
winners

game_id
0        1
1        1
2        0
3        0
4        0
        ..
38867    1
38868    0
38869    0
38870    1
38871    0
Name: winner, Length: 38872, dtype: int64

In [ ]:
#x_n_train.index = [game_id]
#x_n_train.index.name = 'game_id'
 x_n_train['winners'] = winners

In [ ]:
y_train = x_n_train

game_id
0        1
1        1
2        0
3        0
4        0
        ..
38867    1
38868    0
38869    0
38870    1
38871    0
Name: winner, Length: 38872, dtype: int64

In [ ]:
y_train = y_train.values
y_train

array([[  2.,   6.,   1., ...,  17.,  -6.,   1.],
       [  3.,   4.,   2., ...,  15.,  26.,   1.],
       [ 13.,   1.,   0., ...,  71., -27.,   0.],
       ...,
       [ 13.,   1.,   0., ...,  34.,  34.,   0.],
       [  7.,  15.,   4., ...,  10.,  -1.,   1.],
       [  9.,  10.,   6., ...,  53.,  76.,   0.]])

In [ ]:
!pip install bayesian-optimization

In [ ]:
import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import pandas as pd                         # 데이터 분석 라이브러리                     # 계산 라이브러리
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")    

In [ ]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = lgb.LGBMClassifier(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (16, 1024),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------


ValueError: ignored